<a href="https://colab.research.google.com/github/tranhuuan1703/SegmentationImage/blob/main/ModelEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libaries

In [ ]:
! pip install segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 971.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86

In [ ]:
import json
import cv2
import numpy as np

#IMPORTS
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import random

import skimage.draw
import tifffile
import os
import shutil


import torch
from torchvision import datasets
from torch.utils.data import Dataset ,DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import numpy as np
from torch.optim import lr_scheduler
import segmentation_models_pytorch as smp

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

In [ ]:
path_model_unet = "/content/drive/MyDrive/Segmentation/your_model_u_net.pt"
path_model_deeplabv3 = "/content/drive/MyDrive/Segmentation/your_model_deeplabv3.pt"

In [ ]:
# Define transformations
image_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # Assuming grayscale images
    transforms.Lambda(lambda x: x.clamp(0, 1))
])


class CustomDataset_general(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_folder = os.path.join(root_dir, "images")
        self.mask_folder = os.path.join(root_dir, "masks")
        self.image_files = sorted(os.listdir(self.image_folder))
        self.mask_files = sorted(os.listdir(self.mask_folder))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Read image
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        image_gray = image.convert("L")  # Convert to grayscale

        # Read corresponding mask
        mask_name = self.mask_files[idx]
        mask_path = os.path.join(self.mask_folder, mask_name)
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            # Apply transformations
            image_gray = self.transform(image_gray)
            mask = self.transform(mask)

        return image_gray, mask

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [ ]:
model_load = torch.load("/content/drive/MyDrive/Segmentation/model_u_net.pt")

In [ ]:
import time

In [ ]:
root_dir = "/content/drive/MyDrive/Segmentation/segmentation_brain_data/test2"
dataset_test = CustomDataset_general(root_dir, transform=image_transform)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

iou_scores = []
f1_scores = []
f2_scores = []
accuracys = []
recalls = []
time_loads = []
model_load.eval()
for batch, (X, y) in enumerate(test_loader):
    X = X.to(DEVICE, dtype=torch.float32)
    target = y.to(DEVICE, dtype=torch.int16)
    start_time = time.time()
    output = model_load(X)
    output = (output > 0.5).float()
    end_time = time.time()
    tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='multilabel', threshold=0.5)

    # then compute metrics with required reduction (see metric docs)
    iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
    f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
    f2_score = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
    accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
    recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")

    iou_scores.append(iou_score.item())
    f1_scores.append(f1_score.item())
    f2_scores.append(f2_score.item())
    accuracys.append(accuracy.item())
    time_loads.append(end_time - start_time)

In [ ]:
accuracy = sum(accuracys) / len(accuracys)
iou_score = sum(iou_scores) / len(iou_scores)
f1_score = sum(f1_scores) / len(f1_scores)
f2_score = sum(f2_scores) / len(f2_scores)
time_load = sum(time_loads) / len(time_loads)

print("==================Average Metrics U-Net===================")
print("Accuracy:",round(accuracy, 2), '%')
print("IoU Score:", round(iou_score, 2), '%')
print("F1 Score:", round(f1_score, 2), '%')
print("F2 Score:", round(f2_score, 2), '%')
print("Time Load:", round(time_load, 2), 's')

==================Average Metrics U-Net===================
Accuracy: 0.98 %
IoU Score: 0.54 %
F1 Score: 0.64 %
F2 Score: 0.65 %
Time Load: 0.01 s


# evalu model DeepLabV3

In [ ]:
path = "/content/drive/MyDrive/Segmentation/your_model_deeplab_V3.pt"
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'


ACTIVATION = None
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=1,
    classes=1,
    activation=ACTIVATION,
)

model.load_state_dict(torch.load(path))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 137MB/s]


<All keys matched successfully>

In [ ]:
iou_scores = []
f1_scores = []
f2_scores = []
accuracys = []
recalls = []
time_loads = []
model_load.eval()
for batch, (X, y) in enumerate(test_loader):
    X = X.to(dtype=torch.float32)
    target = y.to(dtype=torch.int16)
    start_time = time.time()
    output = model(X)
    output = (output > 0.5).float()
    end_time = time.time()
    tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='multilabel', threshold=0.5)

    # then compute metrics with required reduction (see metric docs)
    iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
    f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
    f2_score = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
    accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
    recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")

    iou_scores.append(iou_score.item())
    f1_scores.append(f1_score.item())
    f2_scores.append(f2_score.item())
    accuracys.append(accuracy.item())
    time_loads.append(end_time - start_time)

In [ ]:
accuracy = sum(accuracys) / len(accuracys)
iou_score = sum(iou_scores) / len(iou_scores)
f1_score = sum(f1_scores) / len(f1_scores)
f2_score = sum(f2_scores) / len(f2_scores)
time_load = sum(time_loads) / len(time_loads)
print("==================Average Metrics DeepLabV3===================")
print("Accuracy:",round(accuracy, 2), '%')
print("IoU Score:", round(iou_score, 2), '%')
print("F1 Score:", round(f1_score, 2), '%')
print("F2 Score:", round(f2_score, 2), '%')
print("Time Load:", round(time_load, 2), 's')

==================Average Metrics DeepLabV3===================
Accuracy: 0.96 %
IoU Score: 0.44 %
F1 Score: 0.56 %
F2 Score: 0.61 %
Time Load: 0.39 s
